# BNNs

In [1]:
from bayesian_models import Pbnn
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from doepy import build
import pickle
from scipy.stats import norm, uniform, lognorm

In [2]:
from LStates.four_branch import four_branch

function = four_branch

def convert_lognormal(mean_ln, std_ln):
    gaussian_param = np.zeros(2)

    SigmaLogNormal = np.sqrt( np.log(1+(std_ln/mean_ln)**2))
    MeanLogNormal = np.log( mean_ln ) - SigmaLogNormal**2/2

    gaussian_param[0] = MeanLogNormal
    gaussian_param[1] = SigmaLogNormal

    return gaussian_param

In [3]:
dim = 2
n_mcs = int(1e6)
x_test = np.random.normal(0, 1, size=(n_mcs, dim))

exp = {'x1': [0, 1, 'normal'],
       'x2': [0, 1, 'normal']}

y_test = function(x_test)
Pf_ref = np.sum( y_test < 0 ) / n_mcs
B_ref= - norm.ppf( Pf_ref )

Pf_ref, B_ref

(0.004371, 2.621982187159241)

Active Train

In [4]:
number_experiments = 1
passive_samples = 20
active_samples = 80
output = 1
layers, archit = 2 , [8,8]  #change archit size if layers are increased
bnn_simulations = 500
training_epochs = 15000
# batch_size = 64
n_mcs = int(1e6)

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '########################################################' )
    
    ActiveTrain_1 = {}

    #PASSIVE TRAINING---------------------------------------------------
    X = np.zeros((passive_samples, dim))
   
    exp_norm = {}
    for var_name in range(dim):
        exp_norm['x'+ str(var_name+1)] = [0.001, 0.999]    #initial design domain for each variable (normal, uniform)
    #Latin hypercube sampling
    Xdoe = build.space_filling_lhs(exp_norm , num_samples = passive_samples)

    for margin in range (0, dim):
        var = 'x' + str (margin + 1)

        if exp[var][2] == 'normal':
            loc_ = exp[var][0]
            scale_ = exp[var][1]
            X[:, margin] = norm.ppf(Xdoe[var], loc=loc_, scale=scale_)

        elif exp[var][2] == 'uniform':
            loc_ = exp[var][0]
            scale_ = exp[var][1]
            X[:, margin] = uniform.ppf(Xdoe[var], loc=loc_, scale=scale_-loc_)

        elif exp[var][2] == 'lognormal':
            xlog_mean = exp[var][0]
            xlog_std = exp[var][1]
            gaussian_param = convert_lognormal(xlog_mean, xlog_std)
            X[:, margin] = lognorm.ppf(Xdoe[var], s=gaussian_param[1], scale=xlog_mean) 

    Y = function(X)

    n_doe = len(X)
    #-------------------------------------------creating bnn
    #setting up the network architecture -----------------------------------
    config = {"n_infeatures": dim,
            "n_outfeatures": output,
            "n_samples": n_doe,
            "learn_all_params": False,  #to learn mean and sigma
            "fixed_param": 0.001} 
    
    ModelName = 'BNN_' + str(n_doe)
    mybnn = Pbnn(config)
    
    mybnn.build_bnn(layers, archit) #----------------------------------------------------------MODEL ARCHITECTURE
    #-------------------------------------------training bnn


    #ACTIVE TRAIN LOOP ---------------------------------------------------
    for active in range(passive_samples, active_samples+passive_samples+1):
        
        batch_size = [np.floor_divide(len(X), 4)][0] + 1  # half of the DoE
#         batch_size = 1

        train_env = {"batch_size": batch_size,
            "learning_rate": 0.001,
            "epochs": training_epochs,
            "callback_patience": 4000,
            "verbose": 0}
        
        print('Training BNN With', active, 'samples' )
        mybnn.train_bnn(X, Y, train_env)

        #-------------------------------------------MC population
        Xtest = np.zeros((int(n_mcs), dim))
        MCinputs_norm= np.random.uniform(0.000001, 0.999999, size=(int(n_mcs), dim))

        for margin in range (0, dim):
            var = 'x' + str (margin + 1)

            if exp[var][2] == 'normal':
                loc_ = exp[var][0]
                scale_ = exp[var][1]
                Xtest[:, margin] = norm.ppf(MCinputs_norm[:, margin], loc=loc_, scale=scale_)

            elif exp[var][2] == 'uniform':
                loc_ = exp[var][0]
                scale_ = exp[var][1]
                Xtest[:, margin] = uniform.ppf(MCinputs_norm[:, margin], loc=loc_, scale=scale_-loc_)

            elif exp[var][2] == 'lognormal':
                xlog_mean = exp[var][0]
                xlog_std = exp[var][1]
                gaussian_param = convert_lognormal(xlog_mean, xlog_std)
                Xtest[:, margin] = lognorm.ppf(MCinputs_norm[:, margin], s=gaussian_param[1], scale=xlog_mean) 

        #-------------------------------------------model predictions over MC population
        print('BNN predictions with MC population...')
        Mean_muY_MC, Stdv_muY_MC, Mean_sigmaY_MC, Stdv_sigmaY_MC = mybnn.modeluq_bnn(Xtest, nsim = bnn_simulations)
        y_mcs = function(Xtest)

        PF = np.sum(Mean_muY_MC < 0) / n_mcs
        B = - norm.ppf( PF )
        PF_ref = np.sum(y_mcs < 0) / n_mcs

        print('PF_ref =', PF_ref, 'PF =', PF, 'and B =',"%.5f" % round(B, 3) ,'-------------------------------')
        print(' ')
        #-------------------------------------------Selecting new training point
        U_f = np.abs(Mean_muY_MC) / Stdv_muY_MC
        U_min = np.argmin(U_f)
        X_new = Xtest[U_min].reshape(-1, dim)
        X = np.concatenate((X, X_new), axis=0)

        Y_new = function(X_new)
        Y = np.concatenate((Y, Y_new), axis=0)

        #-------------------------------------------Saving results
        ActiveTrain_1[ModelName] = mybnn.weights, PF, B
        filename1 = 'Batch_'+ str(experiments+1)+'.sav'
    
        pickle.dump(ActiveTrain_1, open(filename1, 'wb'))


Experiment:  1 ########################################################


C:\Users\Administrator\anaconda3\envs\deep_learning\lib\site-packages\tensorflow_probability\python\layers\util.py:99: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  loc = add_variable_fn(
C:\Users\Administrator\anaconda3\envs\deep_learning\lib\site-packages\tensorflow_probability\python\layers\util.py:109: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  untransformed_scale = add_variable_fn(


Training BNN With 20 samples
Training completed
Minimum loss:  144.68112182617188
BNN predictions with MC population...
PF_ref = 0.004471 PF = 0.0 and B = inf -------------------------------
 
Training BNN With 21 samples
Training completed
Minimum loss:  1864.2239990234375
BNN predictions with MC population...
PF_ref = 0.004368 PF = 2e-06 and B = 4.61100 -------------------------------
 
Training BNN With 22 samples
Training completed
Minimum loss:  85.91220092773438
BNN predictions with MC population...
PF_ref = 0.004523 PF = 0.001874 and B = 2.89900 -------------------------------
 
Training BNN With 23 samples
Training completed
Minimum loss:  76.70196533203125
BNN predictions with MC population...
PF_ref = 0.004521 PF = 0.001453 and B = 2.97800 -------------------------------
 
Training BNN With 24 samples
Training completed
Minimum loss:  76.0871353149414
BNN predictions with MC population...
PF_ref = 0.004399 PF = 0.001531 and B = 2.96100 -------------------------------
 
Traini

Training completed
Minimum loss:  40.752262115478516
BNN predictions with MC population...
PF_ref = 0.004463 PF = 0.001668 and B = 2.93500 -------------------------------
 
Training BNN With 61 samples
Training completed
Minimum loss:  41.19534683227539
BNN predictions with MC population...
PF_ref = 0.004447 PF = 0.001599 and B = 2.94800 -------------------------------
 
Training BNN With 62 samples
Training completed
Minimum loss:  40.68730163574219
BNN predictions with MC population...
PF_ref = 0.00441 PF = 0.001583 and B = 2.95100 -------------------------------
 
Training BNN With 63 samples
Training completed
Minimum loss:  41.25992202758789
BNN predictions with MC population...
PF_ref = 0.004394 PF = 0.001601 and B = 2.94800 -------------------------------
 
Training BNN With 64 samples
Training completed
Minimum loss:  41.24036407470703
BNN predictions with MC population...
PF_ref = 0.004464 PF = 0.001634 and B = 2.94100 -------------------------------
 
Training BNN With 65 sam

In [ ]:
mybnn3 = Pbnn(config)
mybnn3.build_bnn(layers, archit) #----------------------------------------------------------MODEL ARCHITECTURE
mybnn3.train_bnn(X, Y, train_env)

In [ ]:
#-------------------------------------------MC population
# Xtest = np.random.normal(0.0, 1.0, size=(n_mcs, dim))
Xtest = np.zeros((int(n_mcs), dim))
MCinputs_norm= np.random.uniform(0.001, 0.999, size=(int(n_mcs), dim))

for margin in range (0, dim):
    var = 'x' + str (margin + 1)

    if exp[var][2] == 'normal':
        loc_ = exp[var][0]
        scale_ = exp[var][1]
        Xtest[:, margin] = norm.ppf(MCinputs_norm[:, margin], loc=loc_, scale=scale_)

    elif exp[var][2] == 'uniform':
        loc_ = exp[var][0]
        scale_ = exp[var][1]
        Xtest[:, margin] = uniform.ppf(MCinputs_norm[:, margin], loc=loc_, scale=scale_-loc_)

    elif exp[var][2] == 'lognormal':
        xlog_mean = exp[var][0]
        xlog_std = exp[var][1]
        gaussian_param = convert_lognormal(xlog_mean, xlog_std)
        Xtest[:, margin] = lognorm.ppf(MCinputs_norm[:, margin], s=gaussian_param[1], scale=xlog_mean) 

In [ ]:
Mean_muY_MC, Stdv_muY_MC, Mean_sigmaY_MC, Stdv_sigmaY_MC = mybnn3.modeluq_bnn(Xtest, nsim = bnn_simulations)

PF = np.sum(Mean_muY_MC < 0) / n_mcs
B = - norm.ppf( PF )

PF, B